In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ddm_model import DDM, Threshold, Trial

# Base Model (Pure DDM)

In [44]:
ddm_one = DDM(dt=1, drift_rate=0.01, noise_mag=0.5, threshold=20)

In [45]:
ddm_one.threshold.collapse(1000,10,0.008,10)

In [46]:
ddm_one.simulate(trials=10,timesteps=1000)

In [47]:
ddm_one.plot_trajectories()

In [48]:
ddm_one.drift_rate

0.01

In [49]:
ddm_one.reaction_times.mean()

np.float64(362.8)

In [50]:
ddm_one.error_rate

0.3

In [51]:
ddm_one.simulated_results

[('B', 'B'),
 ('A', 'A'),
 ('B', 'B'),
 ('B', 'B'),
 ('B', 'A'),
 ('B', 'A'),
 ('A', 'B'),
 ('B', 'B'),
 ('A', 'A'),
 ('A', 'A')]

Trajectory simulation data:

In [52]:
pd.DataFrame(ddm_one.simulated_trajectories).T

,0,1,2,3,4,5,6,7,8,9
0,-1.343676,0.357112,-0.670801,-1.044780,-0.449762,0.104287,0.936615,-0.130573,-0.535378,-1.187715
1,-1.535113,-0.106565,-1.068733,-0.610150,-0.908823,-0.685338,0.831641,0.053003,-0.575490,-1.020624
2,-2.389684,-0.140349,-0.613021,-1.112766,-0.376713,-0.348830,0.942696,0.143166,-0.457266,-1.366273
3,-2.717031,0.408065,-1.031617,-0.923016,-0.572146,-0.315295,1.264877,-0.034904,-2.022611,-0.834761
4,-3.333476,0.559780,-0.694484,-1.454314,-0.701135,-0.824935,1.400282,-0.033204,-1.013710,-0.783780
...,...,...,...,...,...,...,...,...,...,...
739,NaN,NaN,-8.820107,NaN,NaN,NaN,NaN,NaN,NaN,NaN
740,NaN,NaN,-8.458332,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,NaN,NaN,-9.062200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
742,NaN,NaN,-9.600601,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Biased Model

In [53]:
ddm3 = DDM(dt=1, drift_rate=0.02, noise_mag=0.5, threshold=20,initial_condition=-15)

In [54]:
ddm3.threshold.collapse(1000,10,0.005,8)

In [55]:
ddm3.simulate(trials=10,timesteps=10000)

In [56]:
ddm3.plot_trajectories()

In [57]:
ddm3.simulated_results

[('B', 'B'),
 ('B', 'B'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'B'),
 ('B', 'A'),
 ('B', 'A'),
 ('B', 'B'),
 ('B', 'B'),
 ('B', 'B')]

In [58]:
ddm3.error_rate

0.3

In [59]:
ddm3.reaction_times

array([[497.],
       [ 10.],
       [ 96.],
       [ 16.],
       [ 21.],
       [108.],
       [ 47.],
       [ 13.],
       [ 11.],
       [ 12.]])